In [2]:
import pandas as pd
import numpy as np
import pandas as pd



In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

In [4]:
from tqdm import tqdm

In [ ]:
%run preprocessing.ipynb


In [ ]:
data = master_df

data.shape

In [ ]:
df = pd.DataFrame(data)

# Convert month_id to datetime
def convert_month_id_to_datetime(month_id):
    base_year = 1990
    base_month = 1  # January is month 1
    year = (month_id - 121) // 12 + base_year
    month = (month_id - 121) % 12 + base_month
    return pd.to_datetime({'year': year, 'month': month, 'day': np.ones_like(year)})

df['date'] = convert_month_id_to_datetime(df['month_id'])


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming continuous features need scaling
features_to_scale = ['ged_sb', 'ged_os', 'ged_ns']
scaler = StandardScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])



In [ ]:
def batch_feature_importance(df, target_col='ged_sb', batch_size=20):
    # Define the target and prepare the DataFrame
    y = df[target_col].astype(int)  # Ensure the target variable is suitable for classification
    X = df.drop(target_col, axis=1)  # Drop the target column from the feature set
    
    # Initialize the imputer and scaler
    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()
    
    # Prepare to collect feature importance data
    feature_importances = pd.DataFrame()
    feature_importances['Feature'] = X.columns
    feature_importances['Importance'] = 0
    
    # Calculate feature importance in batches
    n_features = X.shape[1]
    for start in tqdm(range(0, n_features, batch_size)):
        end = start + batch_size
        # Select a batch of features
        batch_features = X.columns[start:end]
        X_batch = X[batch_features]
        
        # Handle missing values and scale the data
        X_batch_imputed = imputer.fit_transform(X_batch)
        X_batch_scaled = scaler.fit_transform(X_batch_imputed)
        
        # Fit the RandomForest classifier
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_batch_scaled, y)
        
        # Store the importances for this batch
        feature_importances.loc[start:end-1, 'Importance'] += clf.feature_importances_

    # Normalize the importance values to sum to 1
    feature_importances['Importance'] /= feature_importances['Importance'].sum()
    
    # Sort features by their importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    
    return feature_importances




In [ ]:

important_features = batch_feature_importance(df, 'ged_sb', 20)
print(important_features.head(50))  



In [ ]:
top_50_features = important_features.head(50)

In [ ]:
# Assuming the DataFrame includes a 'date' and 'priogrid_gid' for sorting and grouping
# Filter the DataFrame to keep only the top 50 features plus the target
df_filtered = df[top_50_features.tolist() + ['ged_sb']]


grouped = df_filtered.sort_values(by='date').groupby('priogrid_gid')
transformer_input = {gid: grp.drop(columns=['priogrid_gid']).values for gid, grp in grouped}



In [ ]:
# Print the shapes of each array in the transformer_input dictionary
for gid, array in transformer_input.items():
    print(f'Shape of array for priogrid_gid {gid}: {array.shape}')


Shape of array for priogrid_gid 62356: (392, 3)
Shape of array for priogrid_gid 79599: (392, 3)
Shape of array for priogrid_gid 79600: (392, 3)
Shape of array for priogrid_gid 79601: (392, 3)
Shape of array for priogrid_gid 80317: (392, 3)
Shape of array for priogrid_gid 80318: (392, 3)
Shape of array for priogrid_gid 80319: (392, 3)
Shape of array for priogrid_gid 80320: (392, 3)
Shape of array for priogrid_gid 80321: (392, 3)
Shape of array for priogrid_gid 80322: (392, 3)
Shape of array for priogrid_gid 80323: (392, 3)
Shape of array for priogrid_gid 80324: (392, 3)
Shape of array for priogrid_gid 80325: (392, 3)
Shape of array for priogrid_gid 80326: (392, 3)
Shape of array for priogrid_gid 80327: (392, 3)
Shape of array for priogrid_gid 80328: (392, 3)
Shape of array for priogrid_gid 80329: (392, 3)
Shape of array for priogrid_gid 80330: (392, 3)
Shape of array for priogrid_gid 80331: (392, 3)
Shape of array for priogrid_gid 80332: (392, 3)
Shape of array for priogrid_gid 81037: (